In [2]:
import tensorrt

In [3]:
import cv2 
import os.path as osp
import os
import glob
from mmdet.apis import inference_detector, init_detector
from mmpose.apis import inference_top_down_pose_model, init_pose_model,process_mmdet_results,vis_pose_result
import mmcv
import numpy as np
import shutil
# import tensorrt as trtz
from Pose.Hrnet import Hrnet
from Pose.Yolov7 import Yolov7


c:\Users\PhuTuan\anaconda3\envs\pythonProject\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\PhuTuan\anaconda3\envs\pythonProject\lib\site-packages\mmcv\__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [4]:
# pose_config = 'mmpose/configs/body/2d_kpt_sview_rgb_img/topdown_heatmap/coco/hrnet_w48_coco_256x192.py'
# pose_checkpoint = 'hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth'
# ###########
# det_config = 'Pose/yolox_s_8x8_300e_coco.py'
# det_checkpoint = 'Pose/yolox_s_8x8_300e_coco_20211121_095711-4592a793.pth'
###############
pose_config = 'Pose/hrnet_w48_coco_256x192.py'
pose_checkpoint = 'Pose/hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth'
###########
det_config = 'Pose/yolox_s_8x8_300e_coco.py'
det_checkpoint = 'Pose/yolox_s_8x8_300e_coco_20211121_095711-4592a793.pth'
##############
# initialize pose model
# pose_model = init_pose_model(pose_config, pose_checkpoint)
# # initialize detector
# det_model = init_detector(det_config, det_checkpoint)
hrnet = Hrnet(engine_path='Pose/Hrnet48_fp32.trt')
hrnet.get_fps()
hrnet.destory()
yolov7 = Yolov7(engine_path='Pose/yolov7_fp16.trt')
yolov7.get_fps()
yolov7.destory()

def inference_image(img,detect:Yolov7,pose:Hrnet):
    det_results = detect.inference(img)
    pose_results = pose.inference_from_bbox(img,det_results)
    return pose_results

def extract_frame(video_path):
    dname = 'temp'
    os.makedirs(dname, exist_ok=True)
    frame_tmpl = osp.join(dname, 'img_{:05d}.jpg')
    cap = cv2.VideoCapture(video_path)
    frame_paths = []
    cnt = 0
    while(cap.isOpened()):
        flag, frame = cap.read()
        if flag:
            frame_path = frame_tmpl.format(cnt + 1)
            frame_paths.append(frame_path)
            frame=cv2.resize(frame,(640,480))
            cv2.imwrite(frame_path, frame)
            cnt += 1
        else: break
    cap.release()
    return frame_paths

def detection_inference(det_config, det_checkpoint ,frame_paths, det_score_thr=0.5,device='cuda' ):
    model = init_detector(det_config, det_checkpoint, device)
    assert model.CLASSES[0] == 'person', ('We require you to use a detector '
                                          'trained on COCO')
    results = []
    print('Performing Human Detection for each frame')
    prog_bar = mmcv.ProgressBar(len(frame_paths))
    for frame_path in frame_paths:
        result = inference_detector(model, frame_path)
        # We only keep human detections with score larger than det_score_thr
        result = result[0][result[0][:, 4] >= det_score_thr]
        results.append(result)
        prog_bar.update()
    return results

def pose_inference(pose_config,pose_checkpoint, frame_paths,image_shape, det_results, device='cuda'):
    model = init_pose_model(pose_config, pose_checkpoint, device)
    print('Performing Human Pose Estimation for each frame')
    prog_bar = mmcv.ProgressBar(len(frame_paths))

    num_frame = len(det_results)
    num_person = max([len(x) for x in det_results])
    if num_person == 0:
        kp = np.zeros((1, num_frame, 17, 3), dtype=np.float32)
        return kp    
    kp = np.zeros((num_person, num_frame, 17, 3), dtype=np.float32)
    for i, (f, d) in enumerate(zip(frame_paths, det_results)):
        # Align input format
        if len(d) == 0: 
            prog_bar.update()
            continue
        d = [dict(bbox=x) for x in list(d) if x[-1] > 0.5]
        pose = inference_top_down_pose_model(model, f, d, format='xyxy')[0]
        vis_ske = vis_pose_result(model,f,pose,dataset=model.cfg.data.test.type,show=False)
        if cv2.waitKey(20)& 0xFF==ord('q'): break
        cv2.imshow('',vis_ske)
        for j, item in enumerate(pose):
            #kp and score (x,y,score) / (widt,heigt,1)
            # normkp = item['keypoints']/(image_shape + (1,)) 
            # kp[j, i] = normkp
            kp[j,i] = item['keypoints']
        prog_bar.update()
    cv2.destroyAllWindows()
    return kp

def pose_extraction(vid,det_config, det_checkpoint,pose_config,pose_checkpoint,label, det_score_thr=0.5,device='cuda'):
    frame_paths = extract_frame(vid)
    det_results = detection_inference(det_config, det_checkpoint ,frame_paths, det_score_thr,device)
    image = cv2.imread(frame_paths[0])
    image_shape = (image.shape[1], image.shape[0])
    pose_results = pose_inference(pose_config,pose_checkpoint, frame_paths,image_shape, det_results, device)
    anno = dict()
    anno['kp'] = pose_results
    anno['img_shape'] = image_shape
    # anno['original_shape'] = image_shape
    anno['total_frames'] = pose_results.shape[1]
    anno['label'] = label
    shutil.rmtree(osp.dirname(frame_paths[0]))
    return anno

def pose_extract(vid,yolov7,hrnet,label,score_threshold=0.5):
    frame_paths = extract_frame(vid)
    prog_bar = mmcv.ProgressBar(len(frame_paths))
    kp = np.zeros(1,len(frame_paths),17,3)
    previous_kp = None
    for idx,frame_path in enumerate(frame_paths):
        img = cv2.imread(frame_path)
        pose_results = inference_image(img,yolov7,hrnet)
        if (pose_results is None):
            kp[0,idx] = previous_kp
            prog_bar.update()
            continue
        kp[0,idx] = pose_results[0]['keypoints']
        previous_kp = pose_results[0]['keypoints']
        prog_bar.update()
    

58.58882945391229 FPS
146.1155189163755 FPS


In [5]:
img = cv2.imread('mmpose/tests/data/coco/000000196141.jpg')
hrnet = Hrnet(engine_path='Pose/Hrnet48_fp32.trt')
hrnet.get_fps()
hrnet.destory()
yolov7 = Yolov7(engine_path='Pose/yolov7_fp16.trt')
yolov7.get_fps()
yolov7.destory()
det = yolov7.inference(img)

58.25808330836472 FPS
118.21029611896593 FPS


In [20]:
a = 1
b = None
if a is None or b is None:
    print('NONE')

NONE


In [6]:
img = cv2.imread('mmpose/tests/data/coco/000000196141.jpg')
det = yolov7.inference(img,0.5)

In [7]:
bbox,score,index = det
person_id = index == 0

In [8]:
bbox_person = bbox[person_id]
score_person = score[person_id]

In [20]:
idx = []
for idx,score in enumerate(score_person):
    if score > 0.5: print(idx,score)
    

0 0.9453125
1 0.890625
2 0.8691406
3 0.77685547
4 0.5756836


In [12]:
idx = []
for idx,score in enumerate(score_person):
    if score > 0.5: print(idx,score)
    

0 0.9453125
1 0.890625
2 0.8691406


In [13]:
index

array([ 0, 35,  0,  0])

In [16]:
index[index==0]

array([0, 0, 0, 0, 0])

In [15]:
pose_results = hrnet.inference_from_bbox(img,det)

In [17]:
pose_results[0]['keypoints']

array([[298.90625   , 111.5625    ,   0.97845697],
       [298.90625   , 105.125     ,   0.93280798],
       [298.90625   , 105.125     ,   0.92875093],
       [311.78125   , 111.5625    ,   0.86448741],
       [331.09375   ,  98.6875    ,   0.82510203],
       [324.65625   , 137.3125    ,   0.81725323],
       [350.40625   , 118.        ,   0.897726  ],
       [292.46875   , 169.5       ,   1.00955188],
       [311.78125   , 130.875     ,   0.79496318],
       [266.71875   , 156.625     ,   0.9611975 ],
       [273.15625   , 143.75      ,   0.88167632],
       [331.09375   , 214.5625    ,   0.79638404],
       [343.96875   , 208.125     ,   0.76659054],
       [324.65625   , 298.25      ,   0.9143784 ],
       [279.59375   , 266.0625    ,   0.91348642],
       [395.46875   , 330.4375    ,   0.870489  ],
       [286.03125   , 343.3125    ,   0.90704066]])

In [3]:
file='C:/Users/PhuTuan/Downloads/Video/Data_fall_171.mp4'

In [4]:
anno = pose_extraction(file, det_config, det_checkpoint,pose_config,pose_checkpoint,label=1)

load checkpoint from local path: Pose/yolox_s_8x8_300e_coco_20211121_095711-4592a793.pth
Performing Human Detection for each frame
[                                                  ] 0/126, elapsed: 0s, ETA:

KeyboardInterrupt: 

In [ ]:

# files=glob.glob('VIDEO_TEST/*.mp4')
# file='VIDEO_TEST/VIDEO11.mp4'
lstfile = []
lstfile.append(sorted(glob.glob('TRAIN/NOT FALL/*.mp4'),key=os.path.getmtime))
lstfile.append(sorted(glob.glob('TRAIN/FALL/*.mp4'),key=os.path.getmtime))
# lstfile.append(sorted(glob.glob('Video_Lying/*.mp4'),key=os.path.getmtime))
# lstfile=sorted(key=os.path.getmtime)
anno_train = []
for index,files in enumerate(lstfile):
    for file in files:
        print('Processing ' + file)
        anno = pose_extraction(file, det_config, det_checkpoint,pose_config,pose_checkpoint,label=index) #LABEL NOT_FALL
        anno_train.append(anno)

In [ ]:
mmcv.dump(anno, 'Data/train1.pkl')

In [ ]:
# file_test='Video_Lying/VIDEO1_4.mp4'
# print('Processing ' + file_test)
# anno = pose_extraction(file_test, det_config, det_checkpoint,pose_config,pose_checkpoint,label=2) #LABEL NOT_FALL
# # anno_train.append(anno)

In [ ]:
# anno

In [ ]:
# for frame in frame_paths:
#     img = cv2.imread(frame)
#     if cv2.waitKey(20)& 0xFF==ord('q'): break
#     cv2.imshow('',img)
# cv2.destroyAllWindows()

In [ ]:
# model = init_pose_model(pose_config, pose_checkpoint, 'cuda')
# print('Performing Human Pose Estimation for each frame')
# prog_bar = mmcv.ProgressBar(len(frame_paths))

# num_frame = len(results)
# num_person = max([len(x) for x in results])
# kp = np.zeros((num_person, num_frame, 17, 3), dtype=np.float32)
# for i, (f, d) in enumerate(zip(frame_paths, results)):
#     # Align input format
#     d = [dict(bbox=x) for x in list(d) if x[-1] > 0.5]
#     pose = inference_top_down_pose_model(model, f, d, format='xyxy')[0]
#     for j, item in enumerate(pose):
#         kp[j, i] = item['keypoints']
#     prog_bar.update()


In [ ]:
# # model = init_detector(det_config, det_checkpoint)
# # assert model.CLASSES[0] == 'person', ('We require you to use a detector '
# #                                         'trained on COCO')
# # results = []
# # print('Performing Human Detection for each frame')
# # prog_bar = mmcv.ProgressBar(len(frame_paths))
# # det_score_thr=0.5
# # for frame_path in frame_paths:
# #     result_det = inference_detector(model, frame_path)
# #     # We only keep human detections with score larger than det_score_thr
# #     result = result_det[0][result_det[0][:, 4] >= det_score_thr]
# #     results.append(result)
# #     prog_bar.update()
# ashape=(2,2)
# bshape=ashape+(1,)
# bshape

In [ ]:
# image = cv2.imread(frame_paths[0])
# image_shape = (image.shape[0], image.shape[1])
# pose_results = pose_inference(pose_config,pose_checkpoint, frame_paths, results)

In [ ]:
# pose_results[0,5]

In [ ]:
# a=pose_results[...,:2]
# a[0,5]

In [ ]:
# anno['kp_score'][0,5]

In [ ]:
# anno['kp'][0,5]

In [ ]:
mmcv.dump(anno, 'Data/train1.pkl')
# import pandas as pds
# data=pds.read_pickle('train.pkl')


In [ ]:
len(anno_train)

In [ ]:
# data[0]

In [ ]:
# type('train.pkl')